NOTEBOOKS REFERENCED FOR THIS ASSIGNMENT:
    https://www.kaggle.com/shlezinger/titanic-classifier-0-7799

What I learned and applied from this notebook:
Originally, I was planning on using OneHotEncoder to handle the categorical variables I wanted to include in my machine learning model. However, when I reached the point of creating "y_pred" I got multiple errors involving the ColumnTransformer tool. I could not figure out on my own what this problem might be. When referencing notebooks to see how others handled this issue I found, like in this notebook referenced above, it was possible to convert these columns to numeric values using np.where and giving dichotomous categorical values either a 1 or a 0.

Additionally, I used the mean age of each data's "Age" column to fill in missing values. 

SCORE GIVEN BY KAGGLE:
0.75837

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import  train_test_split
from sklearn.preprocessing  import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from IPython.display import Image
from sklearn.naive_bayes import GaussianNB

In [ ]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")


In [ ]:
train_data.head()

In [ ]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)


In [ ]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)


In [ ]:
train_data.info()
#objects columns : Name, Sex, Ticket, Cabin, Embarked

In [ ]:
train_data['Sex'].value_counts()


In [ ]:
train_data['Cabin'].value_counts()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

In [ ]:
train_data['Sex'] = np.where(train_data['Sex'] == 'female', 1, 0)
test_data['Sex'] = np.where(test_data['Sex'] == 'female', 1, 0)

In [ ]:
train_data['HasCabin'] = np.where(train_data['Cabin'].isna() == True, 0, 1)


In [ ]:
test_data['HasCabin'] = np.where(test_data['Cabin'].isna() == True, 0, 1)


In [ ]:
train_mean_age = int(train_data['Age'].mean())
train_data['Age'] = np.where(train_data['Age'].isna() == True, train_mean_age, train_data['Age'])

In [ ]:
test_mean_age = int(test_data['Age'].mean())
test_data['Age'] = np.where(test_data['Age'].isna() == True, test_mean_age, test_data['Age'])

In [ ]:
y_train = train_data[['Survived']]
X_train = train_data[['Pclass', 'Age', 'Sex', 'SibSp', 'Parch', 'HasCabin']]

In [ ]:
X_test = test_data[['Pclass', 'Age', 'Sex', 'SibSp', 'Parch', 'HasCabin']]

In [ ]:
X_train.head(20)

In [ ]:
p1 = Pipeline([('imputer', KNNImputer(n_neighbors = 3)),
                     ('scaler', StandardScaler())])
X_train_transformed = p1.fit_transform(X_train)
X_train_transformed

In [ ]:
estimator = []
estimator.append(('RFC', RandomForestClassifier(
    n_estimators=100,
    random_state=42, 
    criterion='entropy',
    max_depth=None, 
    min_samples_split=2)))
estimator.append(('DTC', DecisionTreeClassifier(random_state=3, 
    criterion='entropy',
    splitter='best', 
    max_depth=5, 
    min_samples_split=2)))

In [ ]:
voting_classifier = VotingClassifier(estimators=estimator, voting = 'soft')

In [ ]:
voting_classifier.fit(X_train_transformed, y_train)
y_pred = voting_classifier.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")